## <center>Imputation</center>

Datasets may have missing values, and this can cause problems for many machine learning algorithms. As such, it is good practice to identify and replace missing values for each column in your input data prior to modeling your prediction task. This is called missing data imputation, or imputing for short.

A popular approach for data imputation is to calculate a statistical value for each column (such as a mean) and replace all missing values for that column with the statistic. It is a popular approach because the statistic is easy to calculate using the training dataset and because it often results in good performance.

In this lecture we will review 3 types of imputers:

1. Simple Imputer: the missing values are substituted by user imputed values, these values can be the overall mean of the feature, a specific value, and others. It’s important to highlight that this value is hardcoded by the user o simply calculated with information from the feature.used for order data, where each unique label is mapped to an integer.
2. Middle imputation:  the missing values are substituted using the relationship with other features, divide the feature in different groups, and from these groups calculate the statistical values, such as mean, etc.. for these cases the pandas function .transform is used
3. Iterative Imputer: this is an advanced imputer type uses ML models to predict the missing data



In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

In [2]:
df = pd.read_csv("C:/Users/Lenovo/Desktop/Python/Machine Learning/Preprocessing/Data Sets/Ames_Housing_Data.csv")
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


### Simple Imputer

The most important parameter to take into account is *strategy*, which by default is mean but can take the following arguments:

- “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.
- “median”, then replace missing values using the median along each column. Can only be used with numeric data.
- “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.
- “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.

In [3]:
a = df.select_dtypes("object")

# Categorical features
b = a.isnull().sum()


for ind, val in enumerate(b):
    if val > 0:
        print(b.index[ind], val)

Alley 2732
Mas Vnr Type 23
Bsmt Qual 80
Bsmt Cond 80
Bsmt Exposure 83
BsmtFin Type 1 80
BsmtFin Type 2 81
Electrical 1
Fireplace Qu 1422
Garage Type 157
Garage Finish 159
Garage Qual 159
Garage Cond 159
Pool QC 2917
Fence 2358
Misc Feature 2824


In [4]:
# Taking for example Alley and Misc Feature

imp1 = SimpleImputer(strategy="most_frequent")
imp2 = SimpleImputer(strategy="constant", fill_value="None")

df["Alley"] = imp1.fit_transform(df[["Alley"]])
df["Misc Feature"] = imp2.fit_transform(df[["Misc Feature"]])

In [5]:
a = df.select_dtypes("object")

# Categorical features
b = a.isnull().sum()


for ind, val in enumerate(b):
    if val > 0:
        print(b.index[ind], val)

Mas Vnr Type 23
Bsmt Qual 80
Bsmt Cond 80
Bsmt Exposure 83
BsmtFin Type 1 80
BsmtFin Type 2 81
Electrical 1
Fireplace Qu 1422
Garage Type 157
Garage Finish 159
Garage Qual 159
Garage Cond 159
Pool QC 2917
Fence 2358


In [6]:
a = df.select_dtypes("float")

# Numerical features
b = a.isnull().sum()


for ind, val in enumerate(b):
    if val > 0:
        print(b.index[ind], val)

Lot Frontage 490
Mas Vnr Area 23
BsmtFin SF 1 1
BsmtFin SF 2 1
Bsmt Unf SF 1
Total Bsmt SF 1
Bsmt Full Bath 2
Bsmt Half Bath 2
Garage Yr Blt 159
Garage Cars 1
Garage Area 1


In [7]:
# Taking for example Lot Frontage and Mas Vnr Area

imp1 = SimpleImputer(strategy="mean")
imp2 = SimpleImputer(strategy="median")

df["Lot Frontage"] = imp1.fit_transform(df[["Lot Frontage"]])
df["Mas Vnr Area"] = imp2.fit_transform(df[["Mas Vnr Area"]])

In [8]:
a = df.select_dtypes("float")

# Numerical features
b = a.isnull().sum()


for ind, val in enumerate(b):
    if val > 0:
        print(b.index[ind], val)

BsmtFin SF 1 1
BsmtFin SF 2 1
Bsmt Unf SF 1
Total Bsmt SF 1
Bsmt Full Bath 2
Bsmt Half Bath 2
Garage Yr Blt 159
Garage Cars 1
Garage Area 1


### Middle Imputer

For this type of imputer, it's important to see the relationship between different variables and from this relation impute the missing values. The function .transform form pandas is commonly used.

In [9]:
df = pd.read_csv("C:/Users/Lenovo/Desktop/Python/Machine Learning/Preprocessing/Data Sets/Ames_Housing_Data.csv")
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [10]:
df.corr()

,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
PID,1.000000,-0.001281,-0.096918,0.034868,-0.263147,0.104451,-0.343388,-0.157111,-0.229283,-0.098375,...,-0.051135,-0.071311,0.162519,-0.024894,-0.025735,-0.002845,-0.008260,-0.050455,0.009579,-0.246521
MS SubClass,-0.001281,1.000000,-0.420135,-0.204613,0.039419,-0.067349,0.036579,0.043397,0.002730,-0.060075,...,-0.017310,-0.014823,-0.022866,-0.037956,-0.050614,-0.003434,-0.029254,0.000350,-0.017905,-0.085092
Lot Frontage,-0.096918,-0.420135,1.000000,0.491313,0.212042,-0.074448,0.121562,0.091712,0.222407,0.215583,...,0.120084,0.163040,0.012758,0.028564,0.076666,0.173947,0.044476,0.011085,-0.007547,0.357318
Lot Area,0.034868,-0.204613,0.491313,1.000000,0.097188,-0.034759,0.023258,0.021682,0.126830,0.191555,...,0.157212,0.103760,0.021868,0.016243,0.055044,0.093775,0.069188,0.003859,-0.023085,0.266549
Overall Qual,-0.263147,0.039419,0.212042,0.097188,1.000000,-0.094812,0.597027,0.569609,0.429418,0.284118,...,0.255663,0.298412,-0.140332,0.018240,0.041615,0.030399,0.005179,0.031103,-0.020719,0.799262
Overall Cond,0.104451,-0.067349,-0.074448,-0.034759,-0.094812,1.000000,-0.368773,0.047680,-0.135340,-0.050935,...,0.020344,-0.068934,0.071459,0.043852,0.044055,-0.016787,0.034056,-0.007295,0.031207,-0.101697
Year Built,-0.343388,0.036579,0.121562,0.023258,0.597027,-0.368773,1.000000,0.612095,0.313292,0.279870,...,0.228964,0.198365,-0.374364,0.015803,-0.041436,0.002213,-0.011011,0.014577,-0.013197,0.558426
Year Remod/Add,-0.157111,0.043397,0.091712,0.021682,0.569609,0.047680,0.612095,1.000000,0.196928,0.151790,...,0.217857,0.241748,-0.220383,0.037412,-0.046888,-0.011410,-0.003132,0.018048,0.032652,0.532974
Mas Vnr Area,-0.229283,0.002730,0.222407,0.126830,0.429418,-0.135340,0.313292,0.196928,1.000000,0.301872,...,0.165467,0.143748,-0.110787,0.013778,0.065643,0.004617,0.044934,-0.000276,-0.017715,0.508285
BsmtFin SF 1,-0.098375,-0.060075,0.215583,0.191555,0.284118,-0.050935,0.279870,0.151790,0.301872,1.000000,...,0.224010,0.124947,-0.100455,0.050541,0.095874,0.084140,0.092886,-0.001155,0.022397,0.432914


In [11]:
df.corr()["Year Built"].sort_values(ascending=False).nlargest(5)

Year Built        1.000000
Garage Yr Blt     0.834849
Year Remod/Add    0.612095
Overall Qual      0.597027
SalePrice         0.558426
Name: Year Built, dtype: float64

In [12]:
df["Garage Yr Blt"].isnull().sum()

# As we can see, grage year built has some missing values

159

In [13]:
df["Garage Yr Blt"] = df.groupby(by="Year Built")["Garage Yr Blt"].transform(lambda x: x.fillna(x.mean()))
# In this case mean is chosen, but median, most frequent or other type of value can be selected.

df["Garage Yr Blt"].isnull().sum()
# the missing values decreases dramatically, in this case, the 5 missing could be years that are not processed


5

### Iterative imputation

Iterative imputation is a technique of imputing missing data using regression and classification estimators to model each feature as a function of other features. Each feature is imputed in a round-robin fashion, previous predictions being used in new ones. This process is repeated several times in order to increase the quality of imputation. Compared to simple mean/median/mode imputation, it often can create synthetic values that are closer to real values, at a cost of additional processing time - however, in some cases, simple imputation can give better results.

The iterative imputer has many parameters which are important to tune in, from all of them the most important to highlight are the following:

<ul>
<li>estimator default=BayesianRidge()
The estimator to use at each step of the round-robin imputation. If sample_posterior=True, the estimator must support return_std in its predict method. --> the estimator can be changed to another ML algorithm</li>
<li>initial_strategy{‘mean’, ‘median’, ‘most_frequent’, ‘constant’}, default=’mean’
Which strategy to use to initialize the missing values. Same as the strategy parameter in SimpleImputer.
</li>
<li>
- imputation_order{‘ascending’, ‘descending’, ‘roman’, ‘arabic’, ‘random’}, default=’ascending’
The order in which the features will be imputed. Possible values:
<ul>
<li>
'ascending': From features with fewest missing values to most.</li>
<li>
'descending': From features with most missing values to fewest.</li>
<li>
'roman': Left to right.</li>
<li>
'arabic': Right to left.</li>
<li>
'random': A random order for each round.</li>

</ul>
</li>


</ul>





In [14]:
df = pd.read_csv("C:/Users/Lenovo/Desktop/Python/Machine Learning/Preprocessing/Data Sets/Ames_Housing_Data.csv")
df.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [15]:
a = df.select_dtypes("float")
cols= a.columns


In [16]:
imp_mean = IterativeImputer(random_state=0)

vals = imp_mean.fit_transform(a)
df1 = pd.DataFrame(data=vals, columns=cols)
df1


,Lot Frontage,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath,Garage Yr Blt,Garage Cars,Garage Area
0,141.000000,112.0,639.0,0.0,441.0,1080.0,1.0,0.0,1960.000000,2.0,528.0
1,80.000000,0.0,468.0,144.0,270.0,882.0,0.0,0.0,1961.000000,1.0,730.0
2,81.000000,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1958.000000,1.0,312.0
3,93.000000,0.0,1065.0,0.0,1045.0,2110.0,1.0,0.0,1968.000000,2.0,522.0
4,74.000000,0.0,791.0,0.0,137.0,928.0,0.0,0.0,1997.000000,2.0,482.0
...,...,...,...,...,...,...,...,...,...,...,...
2925,37.000000,0.0,819.0,0.0,184.0,1003.0,1.0,0.0,1984.000000,2.0,588.0
2926,66.124712,0.0,301.0,324.0,239.0,864.0,1.0,0.0,1983.000000,2.0,484.0
2927,62.000000,0.0,337.0,0.0,575.0,912.0,0.0,1.0,1932.286071,0.0,0.0
2928,77.000000,0.0,1071.0,123.0,195.0,1389.0,1.0,0.0,1975.000000,2.0,418.0


In [17]:
df1.isnull().sum()

Lot Frontage      0
Mas Vnr Area      0
BsmtFin SF 1      0
BsmtFin SF 2      0
Bsmt Unf SF       0
Total Bsmt SF     0
Bsmt Full Bath    0
Bsmt Half Bath    0
Garage Yr Blt     0
Garage Cars       0
Garage Area       0
dtype: int64